# ChromaのPythonライブラリをインストール

In [1]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.1 MB/s eta 0:

# ライブラリとデータセットをロード

In [31]:
import chromadb
import pandas as pd

In [ ]:
df = pd.read_csv('data.csv')

In [61]:
df.info()  # Chromaにデータを入れる際、ChromaのIDはstrかつユニークのみ受け付ける

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      100 non-null    object
 1   Item    100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


In [59]:
df.head()

,ID,Item
0,A0,Compact Printer Air Advanced Digital : Situati...
1,A1,Tablet : Discussion loss politics free one tho...
2,A2,Smart Blender Cooker : No situation per.
3,A3,Advanced Router Rechargeable : For force gas e...
4,A4,Portable Mouse Monitor Phone : Feeling back re...


今回はA１のタブレットを探せるようにする

# Chromaの使い方

## Step 1: ChromaのデータをDiskにセーブ

In [75]:
# testのフォルダーが作られ、chroma.sqlite3が作成される
chroma_client = chromadb.PersistentClient(path="test") #　Pathはセーブしたいロケーション　（今回はGoogle Colabなので名前だけ）

## Step 2: Vector　Databaseを作成
create_collection: Collectionを作成  
get_collection: 作成済みのcollectionに接続  
get_or_create_collection: Collectionがある場合は取得、ない場合は新規作成

*Chromaはget_or_create_collectionを使う事を推奨している


In [76]:
# 毎回新しいコレクションを作成しないように、`create_collection` を `get_or_create_collection`に変更してください
collection = chroma_client.get_or_create_collection(name="test")

## Step 3: データをCollectionに加える

In [73]:
collection.add(
    documents=df['Item'].tolist(),
    ids=df['ID'].tolist()
)

データを加えると（testフォルダー内）新しいフォルダーが追加され、Diskに加えられた事が分かる

## Step 4: Queryを書く

In [77]:
results = collection.query(
    query_texts=["Tablet"], # 探したいキーワードを入れる
    n_results=3 # 欲しい数を入れる：　ディフォルトは10個
)
results

{'ids': [['A52', 'A17', 'A1']],
 'embeddings': None,
 'documents': [['Tablet : Many deal community public beyond safe anyone.',
   'Digital Tablet Router Printer Lite : Accept campaign every research test.',
   'Tablet : Discussion loss politics free one thousand.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None, None, None]],
 'distances': [[0.7565003633499146, 1.0221495628356934, 1.1061463356018066]]}

## Step 5:　更新されたVector　Databaseをロードする

In [83]:
del collection  # 今回のデモの為にわざと削除
del chroma_client # 今回のデモの為にわざと削除

In [84]:
chroma_client = chromadb.Client()  # 新規でクラスをインスタンス化する
collection = chroma_client.get_or_create_collection(name="test")

In [85]:
results = collection.query(
    query_texts=["Printer"], # 探したいキーワードを入れる
    n_results=3 # 欲しい数を入れる：　ディフォルトは10個
)
results

{'ids': [['A17', 'A0', 'A38']],
 'embeddings': None,
 'documents': [['Digital Tablet Router Printer Lite : Accept campaign every research test.',
   'Compact Printer Air Advanced Digital : Situation organization these memory much off.',
   'Wireless Tablet Router Printer Wireless Premium Air : North meeting short summer situation positive candidate.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None, None, None]],
 'distances': [[0.9046023488044739, 0.9266445636749268, 1.0031358003616333]]}